In [3]:
import numpy as np
from sklearn.cluster import KMeans
import json

def cluster_split(nodes, node_ids, r_sen = 100, R = 20, max_depth=10, depth=0):
    """
    Hàm phân cụm lặp theo Algorithm 1
    nodes: tọa độ 3D của các node
    node_ids: list id tương ứng của các node
    r_sen: bán kính truyền tải tối đa của node, giả sử là 100m
    R: số lượng node tối đa trong 1 cụm, cho là 20
    max_depth: độ sâu đệ quy tối đa
    """
    center = np.mean(nodes, axis=0) # tâm cụm
    dists = np.linalg.norm(nodes - center, axis=1) # khoảng cách từ tâm đến các node
    if (len(nodes) <= R and np.all(dists <= r_sen)) or depth >= max_depth:
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center
        }]

    # chia cụm bằng KMeans với k = 2
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=20)
    labels = kmeans.fit_predict(nodes)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        clusters += cluster_split(sub_nodes, sub_ids, r_sen, R, max_depth, depth + 1)

    return clusters


def choose_cluster_head(cluster, energies, alpha=0.5):
    # Chọn cluster head là node gần tâm cụm hơn và có năng lượng dư cao hơn
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    dists = np.linalg.norm(nodes - center, axis=1) #khoảng cách từng node đến tâm cụm
    energy_values = np.array([energies[idx] for idx in node_ids]) #năng lượng còn lại của mỗi node
    # Chuẩn hóa
    norm_d = dists / np.max(dists) if np.max(dists) != 0 else dists
    norm_e = energy_values / np.max(energy_values) if np.max(energy_values) != 0 else energy_values
    # Hàm mục tiêu
    f_score = alpha * (1 - norm_e) + (1 - alpha) * norm_d
    ch_index = np.argmin(f_score)
    cluster_head = node_ids[ch_index] # chọn node thỏa mãn làm tâm cụm
    return cluster_head
 


In [ ]:
import os
import json
import numpy as np
import math
import itertools
#from clustering import cluster_split, choose_cluster_head

# =========================
# HÀM TÍNH VS
# =========================
def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    if abs(cos_beta) < 1e-6:
        cos_beta = 1e-6
    inner = (v_f * cos_beta) / v_AUV
    inner = np.clip(inner, -1, 1)
    angle = beta + math.acos(inner)
    v_s = abs(math.cos(angle) * v_AUV / cos_beta)
    return v_s


# =========================
# TÍNH THỜI GIAN DI CHUYỂN
# =========================
def travel_time(path, positions, v_f, v_AUV):
    total_time = 0.0
    O = np.array([0, 0, 0])
    p1, p2 = O, positions[path[0]]
    total_time += np.linalg.norm(p2 - p1) / compute_vs(p1, p2, v_f, v_AUV)
    for i in range(len(path) - 1):
        p1, p2 = positions[path[i]], positions[path[i + 1]]
        total_time += np.linalg.norm(p2 - p1) / compute_vs(p1, p2, v_f, v_AUV)
    p1, p2 = positions[path[-1]], O
    total_time += np.linalg.norm(p2 - p1) / compute_vs(p1, p2, v_f, v_AUV)
    return total_time


# =========================
# TÍNH NĂNG LƯỢNG
# =========================
def compute_energy(best_time, n=4, G=100, L=1024,
                   P_t=1.6e-3, P_r=0.8e-3, P_idle=0.1e-3,
                   DR=4000, DR_i=1e6):
    """Trả về năng lượng tiêu thụ của Member Node và Cluster Head"""
    # Member Node
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = max(0, E_tx_MN + E_idle_MN)

    # Cluster Head (Target Node)
    E_rx_TN = G * P_r * L * n / DR
    E_tx_TN = G * P_t * L * n / DR_i
    E_idle_TN = (best_time - (G * L * n / DR) - (G * L * n / DR_i)) * P_idle
    E_total_TN = max(0, E_rx_TN + E_tx_TN + E_idle_TN)

    return E_total_MN, E_total_TN


# =========================
# GREEDY PATH SELECTION
# =========================
def path_selection(cluster_heads, positions, v_f=1.2, v_AUV=3.0):
    best_time = float("inf")
    best_path = None
    for path in itertools.permutations(cluster_heads):
        t = travel_time(path, positions, v_f, v_AUV)
        if t < best_time:
            best_time = t
            best_path = path
    return best_path, best_time


# =========================
# MÔ PHỎNG NHIỀU CHU KỲ
# =========================
def simulate_auv_pso_energy(data, r_sen=100, R=20,
                            v_f=1.2, v_AUV=3.0,
                            death_threshold=0.9):
    positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
    energies = {d["id"]: 100.0 for d in data}
    total_nodes = len(energies)
    cycle = 0
    total_time_all = 0.0

    while True:
        alive_ids = [nid for nid, e in energies.items() if e > 0]
        alive_count = len(alive_ids)
        dead_ratio = (total_nodes - alive_count) / total_nodes

        if dead_ratio >= death_threshold:
            print(f"\nMạng sập sau {cycle} chu kỳ ({dead_ratio*100:.1f}% nodes chết).")
            break

        cycle += 1
        print(f"\nChu kỳ {cycle}: {alive_count}/{total_nodes} nodes hoạt động")

        # --- Phân cụm ---
        alive_positions = np.array([positions[nid] for nid in alive_ids])
        clusters = cluster_split(alive_positions, alive_ids, r_sen=r_sen, R=R)

        # --- Chọn cluster head ---
        cluster_heads = []
        positions_dict = {}
        for c in clusters:
            ch = choose_cluster_head(c, energies)
            cluster_heads.append(ch)
            positions_dict[ch] = np.mean(c["nodes"], axis=0)

        # --- Greedy path ---
        best_path, best_time = path_selection(cluster_heads, positions_dict, v_f, v_AUV)
        total_time_all += best_time
        print(f"  Đường đi tối ưu: {' → '.join(map(str, best_path))} → O")
        print(f"  Thời gian chu kỳ: {best_time:.2f} s")

        # --- Tính năng lượng tiêu thụ ---
        E_MN, E_CH = compute_energy(best_time)
        print(f"  Năng lượng tiêu thụ: MN={E_MN:.6f} J, CH={E_CH:.6f} J")

        # --- Cập nhật năng lượng ---
        for c in clusters:
            ch = choose_cluster_head(c, energies)
            if energies[ch] > 0:
                energies[ch] = max(0, energies[ch] - E_CH)
            for nid in c["node_ids"]:
                if nid != ch and energies[nid] > 0:
                    energies[nid] = max(0, energies[nid] - E_MN)

        alive_after = len([1 for e in energies.values() if e > 0])
        print(f"  Sau cập nhật: {alive_after} nodes sống.")

    print(f"\nTổng {cycle} chu kỳ, tổng thời gian di chuyển {total_time_all:.2f} s.")


# =========================
# MAIN
# =========================
if __name__ == "__main__":
    with open("D:/Year 4/tiến hóa/project/UWSN_greedy/output_data_kmeans/nodes_20.json", "r") as f:
        data = json.load(f)
    simulate_auv_pso_energy(data)


TypeError: string indices must be integers